In [2]:
import bs4 as BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import re
main_url = 'https://www.basketball-reference.com/players/'
#It was not seen prior that there are no players whos surnames start with x, hence an exception was thrown at letter x. 
#To save time, the available array was copied to another array and the remaining letters were processed 
letters = 'abcdefghijklmnopqrstuvwyz'
letters_list = list(letters)

In [3]:
""" htmldata = urllib.request.urlopen('https://www.basketball-reference.com/players/a/')
bsoup = BeautifulSoup.BeautifulSoup(htmldata,'html.parser')
complete_array = None
Data_list = None """
main_array = None
player_id = 640
for letter in letters_list:
    print(letter)
    try:
        htmldata = urllib.request.urlopen('https://www.basketball-reference.com/players/' + letter +'/')
    except:
        continue
    bsoup = BeautifulSoup.BeautifulSoup(htmldata,'html.parser')
    #a placeholder matrix
    complete_array = np.zeros(shape=(1,1))
    Data_list = None
    for i in range(1,len(bsoup.find_all('tr'))):
        #we collect only players who played after 1980
        if int(bsoup.find_all('tr')[i].find('td',{'data-stat':'year_max'}).get_text())>1980:
            href = bsoup.find_all('tr')[i].find_all('a')[0].attrs['href']
            try:
                player_html = urllib.request.urlopen(main_url + letter + "/" + href.split('/')[-1])
            except:
                #due to errors in the html page, some href do not work hence we skip these
                continue
            player_bsoup = BeautifulSoup.BeautifulSoup(player_html,'html.parser')
            #get draft year
            pattern = r'[0-9]\sNBA Draft'
            draft_year_list = player_bsoup.find_all('a',text = re.compile(pattern))
            if (len(draft_year_list)>0):
                draft_year = draft_year_list[0].get_text().split()[0]
            else:
                draft_year = "Undrafted"
            if Data_list == None:
                Data_list = [item.get_text() for item in player_bsoup.find_all('th') if 'aria-label' in item.attrs.keys()]
                Data_list.insert(0,'Name')

            player_dataarray = np.empty(shape = (len(player_bsoup.find_all('tbody')[0].find_all('tr')),len(Data_list)), dtype = np.object)
            for j in range(0,len(player_bsoup.find_all('tbody')[0].find_all('tr'))):
                data_row = [player_bsoup.find_all('h1',{'itemprop':'name'})[0].get_text().strip()]
                #getting season
                try:
                    #some players wont play some years, at those instances, this code will throw exception
                    #hence we continue to the next year
                    data_row.extend([player_bsoup.find_all('tbody')[0].find_all('tr')[j].find_all('th')[0].get_text()])
                    #adding rest of data    
                    [data_row.append(data) for data in [td.get_text() if td.get_text() != "" else np.nan for td in (player_bsoup.find_all                     ('tbody')[0].find_all('tr')[j].find_all('td'))]]    
                except:
                    continue
                try:     
                    player_dataarray[j,:]  = np.array(data_row)
                except:
                    #any player who doesnt have ['Name','Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%', '2P''2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS'] as there stats are not considered. Majority of players after 1980 will have these stats as these were collected and recorded in this era of basketball
                    break
        else:
            continue
        if i == 1:
            player_id = player_id + 1
            id_array = np.array([player_id]*player_dataarray.shape[0]).reshape(-1,1)
            draft_array = np.array([draft_year]*player_dataarray.shape[0]).reshape(-1,1)
            complete_array = np.c_[id_array,player_dataarray,draft_array]
        else:
            player_id = player_id + 1
            id_array = np.array([player_id]*player_dataarray.shape[0]).reshape(-1,1)
            draft_array = np.array([draft_year]*player_dataarray.shape[0]).reshape(-1,1)
            if complete_array.shape != (1,1):
                complete_array = np.vstack((complete_array,np.c_[id_array,player_dataarray,draft_array]))
            else:
                complete_array = np.c_[id_array,player_dataarray,draft_array]
    if letter == letters_list[0]:
        main_array = complete_array
        print(main_array.shape)
    else:
        main_array = np.vstack((main_array,complete_array))
        print(main_array.shape)
        

a
(1036, 33)
b
(3330, 33)
c
(4927, 33)
d
(6140, 33)
e
(6684, 33)
f
(7239, 33)
g
(8467, 33)
h
(10092, 33)
i
(10259, 33)
j
(11565, 33)
k
(12207, 33)
l
(13108, 33)
m
(15370, 33)
n
(15890, 33)
o
(16344, 33)
p
(17443, 33)
q
(17460, 33)
r
(18683, 33)
s
(20412, 33)
t
(21359, 33)
u
(21433, 33)
v
(21738, 33)
w
(23668, 33)
y
(23751, 33)
z
(23802, 33)


In [4]:
temp = np.zeros(shape = (1,1))
temp.shape == (1,1)


True

In [5]:
atoc_array = main_array

In [6]:
atoc_array

array([[641, 'Alaa Abdelnaby', '1990-91', ..., '0.9', '3.1', '1990'],
       [641, 'Alaa Abdelnaby', '1991-92', ..., '1.9', '6.1', '1990'],
       [641, 'Alaa Abdelnaby', '1992-93', ..., '2.5', '7.7', '1990'],
       ...,
       [3856, 'Ivica Zubac', '2018-19', ..., '2.5', '9.4', '2016'],
       [3856, 'Ivica Zubac', '2019-20', ..., '2.3', '8.3', '2016'],
       [3856, 'Ivica Zubac', '2020-21', ..., '2.4', '7.8', '2016']],
      dtype=object)

In [7]:
final_array = np.vstack((atoc_array, main_array))
final_array

array([[641, 'Alaa Abdelnaby', '1990-91', ..., '0.9', '3.1', '1990'],
       [641, 'Alaa Abdelnaby', '1991-92', ..., '1.9', '6.1', '1990'],
       [641, 'Alaa Abdelnaby', '1992-93', ..., '2.5', '7.7', '1990'],
       ...,
       [3856, 'Ivica Zubac', '2018-19', ..., '2.5', '9.4', '2016'],
       [3856, 'Ivica Zubac', '2019-20', ..., '2.3', '8.3', '2016'],
       [3856, 'Ivica Zubac', '2020-21', ..., '2.4', '7.8', '2016']],
      dtype=object)

In [17]:
Data_list

['PlayerID',
 'Name',
 'Season',
 'Age',
 'Tm',
 'Lg',
 'Pos',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Draft Year']

In [18]:


final_df = pd.DataFrame(data = main_array, columns = Data_list)
final_df

,PlayerID,Name,Season,Age,Tm,Lg,Pos,G,GS,MP,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Draft Year
0,641,Alaa Abdelnaby,1990-91,22,POR,NBA,PF,43,0,6.7,...,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1990
1,641,Alaa Abdelnaby,1991-92,23,POR,NBA,PF,71,1,13.2,...,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,1990
2,641,Alaa Abdelnaby,1992-93,24,TOT,NBA,PF,75,52,17.5,...,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,1990
3,641,Alaa Abdelnaby,1992-93,24,MIL,NBA,PF,12,0,13.3,...,1.0,2.1,3.1,0.8,0.5,0.3,1.1,2.0,5.3,1990
4,641,Alaa Abdelnaby,1992-93,24,BOS,NBA,PF,63,52,18.3,...,1.8,3.0,4.8,0.3,0.3,0.3,1.3,2.6,8.2,1990
5,641,Alaa Abdelnaby,1993-94,25,BOS,NBA,PF,13,0,12.2,...,0.9,2.6,3.5,0.2,0.2,0.2,1.3,1.5,4.9,1990
6,641,Alaa Abdelnaby,1994-95,26,TOT,NBA,PF,54,0,9.4,...,0.7,1.4,2.1,0.2,0.3,0.2,0.8,1.9,4.7,1990
7,641,Alaa Abdelnaby,1994-95,26,SAC,NBA,PF,51,0,9.3,...,0.7,1.4,2.1,0.3,0.3,0.2,0.8,2.0,5.0,1990
8,641,Alaa Abdelnaby,1994-95,26,PHI,NBA,PF,3,0,10.0,...,1.0,1.7,2.7,0.0,0.0,0.0,1.7,0.7,0.7,1990
9,642,Kareem Abdul-Jabbar,1969-70,22,MIL,NBA,C,82,nan,43.1,...,nan,nan,14.5,4.1,nan,nan,nan,3.5,28.8,1969


In [19]:
final_df.to_csv('NBAData.csv',index = False)

In [13]:
main_array

array([[641, 'Alaa Abdelnaby', '1990-91', ..., '0.9', '3.1', '1990'],
       [641, 'Alaa Abdelnaby', '1991-92', ..., '1.9', '6.1', '1990'],
       [641, 'Alaa Abdelnaby', '1992-93', ..., '2.5', '7.7', '1990'],
       ...,
       [3856, 'Ivica Zubac', '2018-19', ..., '2.5', '9.4', '2016'],
       [3856, 'Ivica Zubac', '2019-20', ..., '2.3', '8.3', '2016'],
       [3856, 'Ivica Zubac', '2020-21', ..., '2.4', '7.8', '2016']],
      dtype=object)

['PlayerID',
 'Name',
 'Season',
 'Age',
 'Tm',
 'Lg',
 'Pos',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Draft Year']